# Basic application

This notebook shows you how to use the *basic* Vitis network example that can be generated from this repository.
The design provides network connectivity using User Datagram Protocol (UDP) as the transport protocol.

The assumptions made is this notebook are:
1. You have an Alveo card configured with one of the supported shells
1. You have generated the xclbin file for the *basic* example
1. You have a 100 GbE capable NIC
1. Both the Alveo card and the 100 GbE capable NIC are in the same host
1. Alveo card is connected to the NIC, either directly or with a any network equipment, using any of the interfaces.

Let's have a look at the *basic* design.

There are 4 Kernels:
* CMAC: provides the translation between physical signals to AXI4-Stream interface
* Network layer: provides a bridge between raw Ethernet packets and the application using UDP as transport layer
    * ARP provides translation between MAC and IP addresses
    * ICMP provides ping capabilities
    * The UDP module has a 16-entry table with socket information that needs to be filled in before running
* krnl_mm2s: reads data from memory and packetize it setting tdest and tlast appropriately
* krnl_s2mm: read data from the stream and copy it to memory

![](../img/udp_network_basic.png)

In the followings cells we will:
* Import the necessary pynq and python packages, define current device and xclbin file
* Explore kernels in the design
* Check physical link
* Change Alveo card IP address and ping it
* Configure socket table and populate it to the UDP module
* Create UDP socket in the host
* Allocate Alveo buffers
* Move data from the HOST through the network (NIC) to the Alveo card
* Move data from the HOST through the Alveo card to the network (NIC)
* Free resources

In [1]:
#Uncomment the next line and run to reset the FPGA if it is not taking programming or otherwise misbehaving
#!xbutil reset --device 0000:02:00.1 --force

## Import packages and program FPGA
In this section we need to import the `pynq` and python packages that will be used in the rest of this notebook. We also import the `vnx_utils.py` file with helper functions to set up the vnx examples.

In [2]:
import pynq
import numpy as np
from _thread import *
import threading 
import socket
from vnx_utils import *

In [3]:
# Resource report summary (if available)
import os

rpt = '/home/m2_2/shared/dat480-final/project_split.intf0.xilinx_u55c_gen3x16_xdma_3_202210_1/reports/link/system_estimate_vnx_project_split_if0.xtxt'
if os.path.exists(rpt):
    print("Resource report:", rpt)
    lines = open(rpt, 'r').read().splitlines()
    start = None
    for i, line in enumerate(lines):
        if line.strip() == 'Area Information':
            start = i
            break
    if start is None:
        print("Area Information section not found")
    else:
        first_sep = None
        second_sep = None
        for i in range(start + 1, len(lines)):
            if lines[i].startswith('----'):
                first_sep = i
                break
        if first_sep is not None:
            for i in range(first_sep + 1, len(lines)):
                if lines[i].startswith('----'):
                    second_sep = i
                    break
        if first_sep is None or second_sep is None:
            print("Area Information table not found")
        else:
            table = lines[start:second_sep + 1]
            for line in table:
                print(line)

            # Parse usage and compute percentage within this table
            data_rows = [l for l in table if l.startswith('krnl_proj_split_0')]
            totals = {'FF': 0, 'LUT': 0, 'DSP': 0, 'BRAM': 0, 'URAM': 0}
            parsed = []
            for row in data_rows:
                parts = row.split()
                if len(parts) >= 9:
                    ff, lut, dsp, bram, uram = map(int, parts[-5:])
                    totals['FF'] += ff
                    totals['LUT'] += lut
                    totals['DSP'] += dsp
                    totals['BRAM'] += bram
                    totals['URAM'] += uram
                    parsed.append((parts[2], ff, lut, dsp, bram, uram))

            if sum(totals.values()) > 0:
                print("Resource share within krnl_proj_split table:")
                for name, ff, lut, dsp, bram, uram in parsed:
                    def pct(v, t):
                        return 0.0 if t == 0 else (100.0 * v / t)
                    print("{}: FF {:.2f}%, LUT {:.2f}%, DSP {:.2f}%, BRAM {:.2f}%, URAM {:.2f}%".format(
                        name, pct(ff, totals['FF']), pct(lut, totals['LUT']), pct(dsp, totals['DSP']),
                        pct(bram, totals['BRAM']), pct(uram, totals['URAM'])
                    ))
            else:
                print("No data rows found for krnl_proj_split_0")
else:
    print("Resource report not found:", rpt)

Resource report: /home/m2_2/shared/dat480-final/project_split.intf0.xilinx_u55c_gen3x16_xdma_3_202210_1/reports/link/system_estimate_vnx_project_split_if0.xtxt
Area Information
Compute Unit       Kernel Name      Module Name                                      FF    LUT    DSP  BRAM  URAM
-----------------  ---------------  -----------------------------------------------  ----  -----  ---  ----  ----
krnl_proj_split_0  krnl_proj_split  input_split_Pipeline_VITIS_LOOP_39_2             148   2668   0    0     0
krnl_proj_split_0  krnl_proj_split  input_split                                      877   2812   0    0     0
krnl_proj_split_0  krnl_proj_split  matcher_engine_1720_16_Pipeline_init_state       13    63     0    0     0
krnl_proj_split_0  krnl_proj_split  matcher_engine_1720_16_Pipeline_pattern_loop     118   22295  0    19    0
krnl_proj_split_0  krnl_proj_split  matcher_engine_1720_16_Pipeline_reset_state      13    63     0    0     0
krnl_proj_split_0  krnl_proj_split  matc

We also need to define the current device, only if there is more than one Alveo card on the host. First let's check how many devices are available.

In [4]:
for i in range(len(pynq.Device.devices)):
    print("{}) {}".format(i, pynq.Device.devices[i].name))

0) xilinx_u55c_gen3x16_xdma_base_3


* If there are more than one Alveo card available, you should pass the `device` argument to the `pynq.Overlay` class.
* The xclbin variable should point to the `xclbin` file for the *basic* example

In [5]:
currentDevice = pynq.Device.devices[0]
xclbin = '/home/m2_2/shared/dat480-final/project_split.intf0.xilinx_u55c_gen3x16_xdma_3_202210_1/vnx_project_split_if0.xclbin'
ol = pynq.Overlay(xclbin,device=currentDevice)

## Explore kernels in the design
This design was built for both interfaces. Therefore, we will see each kernel repeated twice. One for the interface 0 and another for the interface 1

In [6]:
ol.ip_dict

{'cmac_0': {'phys_addr': 12582912,
  'addr_range': 8192,
  'type': 'xilinx.com:kernel:cmac_0:1.0',
  'hw_control_protocol': 'ap_ctrl_none',
  'fullpath': 'cmac_0',
  'registers': {'gt_reset_reg': {'address_offset': 0,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 0},
   'reset_reg': {'address_offset': 4,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 1},
   'mode': {'address_offset': 8,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 2},
   'conf_tx': {'address_offset': 12,
    'access': 'read-write;',
    'size': 32,
    'host_size': 4,
    'description': 'OpenCL Argument Register',
    'type': 'uint',
    'id': 3},
   'conf_rx': {'address_offset': 20,
    'access': 'read-write;',
    'size': 32,
    'hos

## Check physical link
After the dynamic region of the Alveo card is programmed with the basic example we can start interacting with the design.

Let's check if the Alveo card has detected link with the network equipment. To do so, we will use one of the helper functions `link_status`

In [7]:
print("Link interface 0 {}".format(ol.cmac_0.link_status()))

Link interface 0 {'cmac_link': True}


## Change Alveo card IP address and ping it
By defaul the Alveo IP address is `192.168.0.5` and MAC address is `00:0A:35:02:9D:E5`. Let's change it to `192.168.100.2`, in this example we are using interface 0

After the IP address is changed, we can ping the Alveo card. The first attempts will fail but the remaining should work.

**Make sure you have configured the IP address of the 100 GbE capable NIC to be in the same subnetwork as the Alveo card**

In [8]:
alveo_ipaddr = '192.168.100.2'
print(ol.networklayer_0.set_ip_address(alveo_ipaddr, debug=True))

{'HWaddr': '00:0a:35:02:9d:02', 'inet addr': '192.168.100.2', 'gateway addr': '192.168.100.1', 'Mask': '255.255.255.0'}


* Check 100 GbE capable NIC configuration

In [9]:
!ip addr show ens1f0np0

3: ens1f0np0: <BROADCAST,MULTICAST,UP,LOWER_UP> mtu 1500 qdisc mq state UP group default qlen 1000
    link/ether 0c:42:a1:ef:fd:96 brd ff:ff:ff:ff:ff:ff
    altname enp1s0f0np0
    inet 192.168.100.1/30 scope global ens1f0np0
       valid_lft forever preferred_lft forever
    inet6 fe80::e42:a1ff:feef:fd96/64 scope link 
       valid_lft forever preferred_lft forever


* Run ping to get ARP set up
Some of the attempts will fail but the remaining should work.

In [10]:
!ping -c 5 $alveo_ipaddr -I ens1f0np0

PING 192.168.100.2 (192.168.100.2) from 192.168.100.1 ens1f0np0: 56(84) bytes of data.

--- 192.168.100.2 ping statistics ---
5 packets transmitted, 0 received, 100% packet loss, time 4076ms



## Configure socket table and populate it to the UDP module

In this section we will configure the socket table in software and populate it to the UDP module in the Alveo card. 
1. Define a couple of connections 
1. The socket table is populated to the UDP module in the Alveo card using the `.populate_socket_table()` helper function

In [ ]:
sw_ip = '192.168.100.1'
host_port = 38746
fpga_port = 62781
ol.networklayer_0.sockets[0] = (sw_ip, host_port, fpga_port, True)

ol.networklayer_0.populate_socket_table(debug=True)

## Create UDP socket in the host
In this part we will open an UDP socket in the host (software) to be able to communicate with the Alveo card through the network.

We will use the python socket API to do so, the socket will be binded to the port `38746`

In [ ]:
SW_PORT = ol.networklayer_0.sockets[0]['theirPort']
sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM) # UDP
sock.bind(('', SW_PORT))
sock.settimeout(2.0)

## Allocate Alveo buffers

We need to allocate the buffers on the global memory of the Alveo card to be able to pull and push data from them. We also will initialize the the sending buffer, `mm2s_buf`, with random data. This random data will be sent later to the network.

1. Define alias for the application kernlels (`mm2s` and `s2mm`)
1. Define size and shape of the buffers
1. Allocate the buffers
1. Initialize sending buffer with random data

In [ ]:
krnl = ol.krnl_proj_split_0
# The design has no active memory banks; start via control register
krnl.register_map.CTRL.AUTO_RESTART = 1
krnl.register_map.CTRL.AP_START = 1


## Move data from the HOST through the network (NIC) to the Alveo card

In this section we will write data to the socket, which will send such data from the host to the Alveo card using the network.

Start streaming to memory mapped kernel, we need to specify how much data to expect from the network.

In [ ]:
payload = b"XXXXUSERYYYYPONGZZZZr00t"

# Drain stale packets before correctness test
while True:
    try:
        sock.recvfrom(2048)
    except socket.timeout:
        break

sock.sendto(payload, (alveo_ipaddr, fpga_port))

data = None
addr = None
for _ in range(3):
    try:
        data, addr = sock.recvfrom(2048)
        break
    except socket.timeout:
        pass

if data is None:
    print("No response received (timeout).")
else:
    ids = [int.from_bytes(data[i:i+4], 'little')
           for i in range(0, len(data), 4)
           if i + 4 <= len(data)]
    expected = {10, 24, 36}
    missing = expected - set(ids)
    ok = (len(missing) == 0)
    print("Received {} bytes from {}".format(len(data), addr))
    print("Correctness:", "PASS" if ok else "FAIL")
    if not ok:
        print("Missing IDs:", sorted(missing))
    print("Match IDs:", ids)

In [ ]:
# Throughput test: send many UDP packets and measure Tx rate
import time

num_pkts = 1000
pkt_size = 1472
payload_thr = b'A' * pkt_size

start_ts = time.time()
for _ in range(num_pkts):
    sock.sendto(payload_thr, (alveo_ipaddr, fpga_port))
end_ts = time.time()

elapsed = end_ts - start_ts
bytes_sent = num_pkts * pkt_size
if elapsed > 0:
    gbps = (bytes_sent * 8.0) / (elapsed * 1e9)
else:
    gbps = 0.0

print("Sent bytes={}, elapsed_s={:.6f}, tx_gbps={:.3f}".format(bytes_sent, elapsed, gbps))

Initialize a buffer with random data, define the packet size and compute how many packets we need to send to transmit the whole buffer to the network. Write the data from the buffer into the socket in `BYTES_PER_PACKET` chunks.

In [ ]:
# sock.close()
# pynq.Overlay.free(ol)

* Wait for the s2mm kernel to receive all the data
* Move data from global memory to HOST memory
* Compare what was sent against what was received and print out result

## Move data from the HOST through the Alveo card to the network (NIC)

We need to create a new thread to read data from the socket, this mainly because there will be multiple packets. This is done in the `socket_receive_threaded` function

* Copy random data to Alveo global memory
* Acquire thread 
* Launch threaded function
* Start kernel memory mapped to stream kernel, this will start sending UDP packets from the Alveo card to the NIC
* Threaded function will start receiving data and it will print out result once ALL data was collected

## Free resources
Delete buffers and free Alveo card

------------------------------------------
Copyright (c) 2020-2021, Xilinx, Inc.